# Policy and Value Iteration
source: [Policy and Value Iteration PDF](https://www.theschool.ai/wp-content/uploads/2018/09/policy_and_value_iteration.pdf)

### Lecture Notes
Now that we are familiar with the basics, its time to move forward. Let us revisit the same problem we
discussed last week. Robotic vacuum cleaner famously known as Roomba is a machine that cleans
the floor. Roomba needs to start, clean, avoid obstacles and find the charging station. These 4 states
describe the possible positions of the robot and the action describes the direction of motion. The
robot can move left/right/up/down. The first (Battery Full) and the final (Charging) states are the
terminal states. The goal is to find an optimal policy that maximizes the return from any initial states.

Let us use OpenAI gym and try to solve this problem. FrozenLake-v0 environment will be a best fit for
this problem. The reward and next state models are stochastic $p(r_t+1 | s_t , a_t ), p(s_t+1 | s_t , a_t
)$ for this environment. To solve this problem, we should find an optimal policy that can reach the goal
more than **70%** of the times. (Please try to come up with a solution before looking at the solutions) 

In [5]:
# Brute Force Method
'''
We got 16 states and 4 possible moves that gives us 4^16 = 4294967296 possible policies to choose
from. It is a computationally intensive task to evaluate all of them so we are going to choose few
cases randomly and select the best among them
'''
import numpy as np
import time
import gym
"""
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI gym env.
        render: boolean to turn rendering on/off.
"""
# Execution
def execute(env, policy, episode_length=100, render=False):
    total_reward = 0
    start = env.reset()
    for t in range(episode_length):
        if render:
            env.render()
        action = policy[start]
        start, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
    return total_reward


# Evaluation
def evaluatePolicy(env, policy, n_episodes=100):
    total_reward = 0.0
    for _ in range(n_episodes):
        total_reward += execute(env, policy)
    return total_reward / n_episodes


# Funtion for a random policy
def gen_random_policy():
    return np.random.choice(4, size=((16)))


# Main
if __name__ == '__main__':
    env = gym.make('FrozenLake-v0')

    ## Policy Search
    n_policies = 1000
    start_time = time.time()
    policy_set = [gen_random_policy() for _ in range(n_policies)]
    policy_score = [evaluatePolicy(env, p) for p in policy_set]
    end_time = time.time()
    print("Best score = {:0.2f}. Time taken = {:4.4f} seconds".format(np.max(policy_score), end_time - start_time))

Best score = 0.54. Time taken = 7.7903 seconds


The above script searches the environment for best policy in a random set of 1000 solutions and evaluates them. The best policy score we get is **0.54** in **7.7903** seconds. It means that the chance of agent reaching the goal is **54%**. It is not even close to our goal. Random search does not work well for complex problems where the search space is huge.

---

**_How can we improve this?_**

The goal of the Roomba is to pick the best policy that will maximize the total rewards received in the environment:
- t_0: Roomba observes the environment state s_0 and picks an action a_0, then upon performing its action, environment state becomes s_1 and the Roomba receives a reward r_1.
- t_1: Roomba observes the environment state s_1 and picks an action a_1, then upon performing its action, environment state becomes s_2 and the Roomba receives a reward r_2.
- t_2: Roomba observes the environment state s_2 and picks an action a_2, then upon performing its action, environment state becomes s_3 and the Roomba receives a reward r_3.

So the total reward received by the Roomba in response to the actions selected by its policy is going to be:


$$ Total Reward = r_1 + r_2 + r_3 + r_4 + r_5 + ...$$

However, it is common to use a discount factor to give higher weight if the rewards are nearby and lower weight if the rewards are far in the future:

$$Total Discounted Reward = r_1 + \gamma{r_2} + \gamma^2{r_3} + \gamma^3{r_4} + \gamma^4{r_5} + ...$$

$$	\sum_{i=1}^{T} \gamma^{i-1}{r_i} $$

where T is the length of the episode. It can be infinity if there is huge length for the episode.

**_Why discount factor?_**

Using a discount favor prevents the total reward from going to infinity (because $0 <= \gamma <= 1$). It also models the agent behavior when the agent prefers intermediate rewards than rewards that are potentially received later in the future.

In our example, Roomba can take two paths to reach its goal state:
- One if longer but gives higher reward
- There is a shorter path with a smaller reward
We need the Roomba to cover and clean maximum distance before it reaches the goal. By adjusting the $\gamma$ value, we can control which path the Roomba should prefer to take.

**Value Function**  
A popular term in Reinforcement Learning and denoted as **V(s)**, it represents how good a state is for an agent to be in.

Amongst all the possible value functions, there will be an optimal value function that has higher value than other functions for all states. This is denoted as **V*(s)**.

The optimal policy $\pi^*$ is the policy that corresponds to the optimal value function.

**Q Function**  
Q is a function of a state-action pair and returns real value, "**Q: S x A --> R**". An optimal Q-function **Q*(s,a)** means the expected total reward received by an agent starting in a state **s** and picks an action **a** will then move optimally forward. In other words, this indicates how good it is for an agent to pick action **a** while in state **s**.

Since **V*(s)** is the maximum expected total reward when starting from state **s**, it will be the maximum of **Q*(s,a)** over all possible actions.

When a known optimal Q-function **Q*(s,a)** exists, the optimal policy can be attained by choosing the action **a** that declares the maximum **Q*(s,a)** for a particular state **s**.

Bellman equation using Dynamic Programming gives a recursive definition for the optimal Q-function. The **Q*(s,a)** equals the summation of immediate reward after performing action **a** while in state **s** and the discounted expected future reward after transition to a next state **s'**.

**Value Iteration**  
Value Iteration computes the optimal state value function by iteratively improving the estimate of **V(s)**. The algorithm initializes **V(s)** to arbitrary random values when it repeatedly updates the **Q(s,a)** and **V(s)** values until they converge. Value iteration is guaranteed to converge to the optimal values.

In [18]:
# Let's take the above problem and try to solve it using the Value Iteration method.
import numpy as np
import gym
import time
"""
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI Gym env.
            env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment.
            env.nA is a number of actions in the environment.
        gamma: Gamma discount factor.
        render: boolean to turn rendering on/off.
"""
# Executes an episode
def execute(env, policy, gamma=0.1, render=False):
    start = env.reset()
    total_reward = 0
    step_index = 0
    while True:
        if render:
            env.render()
        start, reward, done, _ = env.step(int(policy[start]))
        total_reward += (gamma ** step_index * reward)
        step_index += 1
        if done:
            break
    return total_reward


# Evaluates a policy by running it n times. returns: average total reward
def evaluatePolicy(env, policy, gamma=1.0, n=100):
    scores = [
        execute(env, policy, gamma=gamma, render=False)
        for _ in range(n)
    ]
    return np.mean(scores)


# Choosing the policy given a value-function
def calculatePolicy(v, gamma=1.0):
    policy = np.zeros(env.env.nS)
    for s in range(env.env.nS):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy


# Value Iteration Algorithm
def valueIteration(env, gamma=1.0):
    value = np.zeros(env.env.nS) # initialize value-function
    max_iterations = 10000
    eps = 1e-20
    for i in range(max_iterations):
        previous_value = np.copy(value)
        for s in range(env.env.nS):
            q_sa = [
                sum([p * (r + previous_value[s_])
                     for p, s_, r, _ in env.env.P[s][a]])
                for a in range(env.env.nA)
            ]
            value[s] = max(q_sa)
        if (np.sum(np.fabs(previous_value - value)) <= eps):
            print('Value-iteration converged at # {}.'.format(i + 1)) 
            break
    return value


# Main
if __name__ == '__main__':
    gamma = 1.0
    env = gym.make('FrozenLake-v0')
    optimal_value = valueIteration(env, gamma)
    start_time = time.time()
    policy = calculatePolicy(optimal_value, gamma)
    policy_score = evaluatePolicy(env, policy, gamma, n=1000)
    end_time = time.time()
    # why np.mean? typo? Should it not be np.max?
    print("Best score = {:0.2f}. Time taken = {:4.4f} seconds".format(np.mean(policy_score), end_time - start_time))

Value-iteration converged at # 1373.
Best score = 0.73. Time taken = 0.3576 seconds


---

**Policy Iteration:**
The Value-Iteration algorithm keeps improving the value function at each iteration until the Value function converges. Since the agent only cares about finding the optimal policy, sometimes the optimal policy will converge before the Value-Function. Therefore, we have another algorithm called Policy-Iteration. Instead of repeatedly improving the Value-Function estimate, it will re-define the policy at each step and compute the value according to this new policy until the policy converges. Policy Iteration is also guaranteed to converge to the optimal policy and it often takes less iterations to converge than the Value-Iteration algorithm

In [21]:
# Let's take the above problem and try to solve it using the Policy Iteration method.
import numpy as np
import gym
import time
"""
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI Gym env.
            env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment.
            env.nA is a number of actions in the environment.
        gamma: Gamma discount factor.
        render: boolean to turn rendering on/off.
"""
# Executes an episode
def execute(env, policy, gamma=0.1, render=False):
    start = env.reset()
    total_reward = 0
    step_index = 0
    while True:
        if render:
            env.render()
        start, reward, done, _ = env.step(int(policy[start]))
        total_reward += (gamma ** step_index * reward)
        step_index += 1
        if done:
            break
    return total_reward


# Evaluates a policy by running it n times. returns: average total reward
def evaluatePolicy(env, policy, gamma=1.0, n=100):
    scores = [
        execute(env, policy, gamma=gamma, render=False)
        for _ in range(n)
    ]
    return np.mean(scores)


# Extract the policy given a value-function
def extractPolicy(v, gamma=1.0):
    policy = np.zeros(env.env.nS)
    for s in range(env.env.nS):
        q_sa = np.zeros(env.env.nA)
        for a in range(env.env.nA):
            q_sa[a] = sum([p * (r + gamma * v[s_]) 
                           for p, s_, r, _ in env.env.P[s][a]]
                         )
        policy[s] = np.argmax(q_sa)
    return policy

# Iteratively calculates the Value-Function under Policy.
def calculatePolicyValue(env, policy, gamma=1.0):
    value = np.zeros(env.env.nS)
    eps = 1e-10
    while True:
        previous_value = np.copy(value)
        for states in range(env.env.nS):
            policy_a = policy[states]
            value[states] = sum([p * (r + gamma * previous_value[s_]) 
                                 for p, s_, r, _ in env.env.P[states][policy_a]]
                               )
        if (np.sum((np.fabs(previous_value - value))) <= eps):
            # value converged
            break
    return value


# Policy Iteration Algorithm
def policyIteration(env, gamma=1.0):
    policy = np.random.choice(env.env.nA, size=(env.env.nS)) # initialize a random policy
    max_iterations = 1000
    gamma = 1.0
    for i in range(max_iterations):
        old_policy_value = calculatePolicyValue(env, policy, gamma)
        new_policy = extractPolicy(old_policy_value, gamma)
        if (np.all(policy == new_policy)):
            print('Policy Iteration converged at {}'.format(i + 1))
            break
        policy = new_policy
    return policy


# Main
if __name__ == '__main__':
    env = gym.make('FrozenLake-v0')
    start_time = time.time()
    optimal_policy = policyIteration(env, gamma=1.0)
    scores = evaluatePolicy(env, optimal_policy, gamma=1.0)
    end_time = time.time()
    print("Best score = {:0.2f}. Time taken = {:4.4f} seconds".format(np.max(scores), end_time - start_time))

Policy Iteration converged at 4
Best score = 0.77. Time taken = 0.1275 seconds


---

**Value-Iteration vs. Policy-Iteration:**
Both Value-Iteration and Policy Iteration Algorithms can be used for _offline planning_ where the agent is assumed to have prior knowledge about the effects of its actions on the environment (they assume the MDP model is known). Comparing with each other, Policy-Iteration is computationally efficient as it often takes considerably fewer number of iterations to converge although each iteration is more computationally expensive.

---

#### Credits
- [Deep Reinforcement Learning Slide](http://learning.mpi-sws.org/mlss2016/slides/2016-MLSS-RL.pdf)
- [Reinforcement Learning: An Introduction 2nd Ed In Progress](http://incompleteideas.net/book/bookdraft2018jan1.pdf)
- [Deep RL Course - Value Function Methods](http://rail.eecs.berkeley.edu/deeprlcourse/static/slides/lec-7.pdf)
- [OpenAI Gym - FrozenLake-v0 Algorithm](https://gym.openai.com/evaluations/eval_4VyQBhXMRLmG9y9MQA5ePA/)
- [Moustafa F. Alzantot](http://web.cs.ucla.edu/~malzantot/)
- [planetB Syntax Highlight Code in Word Documents](http://www.planetb.ca/syntax-highlight-word)